In [1]:
SANDBOX_NAME = ''# Sandbox Name
DATA_PATH = "/data/sandboxes/" + SANDBOX_NAME + "/data/"



# Spark ML

Cargamos un dataset con información de la distribución de los píxeles para cada una de las letras del alfabeto escritas en mayúsculas. El objetivo será predecir si el caracter en cuestión es una vocal o una consonante. En el proceso se aprenderán los pasos generales a seguir para solucionar un problema de este tipo con Spark ML.




### Crear SparkSession

In [2]:
# Respuesta

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()



### Importar librerias

In [3]:
# Respuesta

import pandas as pd



### Cargar datos y comprobar schema

In [4]:
# Respuesta

letters = spark.read.csv(DATA_PATH+'data/letter.txt', sep=',', header=True, inferSchema=True)

In [5]:
# Respuesta

letters.printSchema()

root
 |-- x_box_integer: integer (nullable = true)
 |-- y_box_integer: integer (nullable = true)
 |-- width_integer: integer (nullable = true)
 |-- high_integer: integer (nullable = true)
 |-- onpix_integer: integer (nullable = true)
 |-- x_bar_integer: integer (nullable = true)
 |-- y_bar_integer: integer (nullable = true)
 |-- x2bar_integer: integer (nullable = true)
 |-- y2bar_integer: integer (nullable = true)
 |-- xybar_integer: integer (nullable = true)
 |-- x2ybr_integer: integer (nullable = true)
 |-- xy2br_integer: integer (nullable = true)
 |-- x_ege_integer: integer (nullable = true)
 |-- xegvy_integer: integer (nullable = true)
 |-- y_ege_integer: integer (nullable = true)
 |-- yegvx_integer: integer (nullable = true)
 |-- class: string (nullable = true)





### Crear nueva variable objetivo

La variable objetivo ahora mismo es cada una de las letras del abecedario en mayúscula. Se crea nueva variable con el nombre 'flag' que tome el valor 1 si se trata de una vocal y 0 en caso contrario, para convertir el problema en un problema de clasificación binaria.

In [6]:
# Respuesta

letters.groupBy('class').count().orderBy('class').show(500)

+-----+-----+
|class|count|
+-----+-----+
|    A|  789|
|    B|  766|
|    C|  736|
|    D|  805|
|    E|  768|
|    F|  775|
|    G|  773|
|    H|  734|
|    I|  755|
|    J|  747|
|    K|  739|
|    L|  761|
|    M|  792|
|    N|  783|
|    O|  753|
|    P|  803|
|    Q|  783|
|    R|  758|
|    S|  748|
|    T|  796|
|    U|  813|
|    V|  764|
|    W|  752|
|    X|  787|
|    Y|  786|
|    Z|  734|
+-----+-----+



In [7]:
# Respuesta

import pyspark.sql.functions as F
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType

letters = letters.withColumn('tag', col('class').isin('A', 'E', 'I', 'O', 'U').cast('double'))
letters.select('class', 'tag').distinct().show(500)

+-----+---+
|class|tag|
+-----+---+
|    X|0.0|
|    L|0.0|
|    A|1.0|
|    B|0.0|
|    R|0.0|
|    G|0.0|
|    Y|0.0|
|    N|0.0|
|    F|0.0|
|    E|1.0|
|    I|1.0|
|    K|0.0|
|    M|0.0|
|    D|0.0|
|    V|0.0|
|    S|0.0|
|    Z|0.0|
|    U|1.0|
|    Q|0.0|
|    W|0.0|
|    H|0.0|
|    J|0.0|
|    T|0.0|
|    O|1.0|
|    C|0.0|
|    P|0.0|
+-----+---+





### Primeros pasos

Primeros pasos: nulos y vector assembler



Empezaremos con la comprobación y tratamiento de nulos.

In [8]:
# Respuesta

for column in letters.columns:
    print('column: {} Nulls: {}'.format(column, letters.filter(col(column).isNull()).count()))

column: x_box_integer Nulls: 0
column: y_box_integer Nulls: 0
column: width_integer Nulls: 0
column: high_integer Nulls: 0
column: onpix_integer Nulls: 0
column: x_bar_integer Nulls: 0
column: y_bar_integer Nulls: 0
column: x2bar_integer Nulls: 0
column: y2bar_integer Nulls: 0
column: xybar_integer Nulls: 0
column: x2ybr_integer Nulls: 0
column: xy2br_integer Nulls: 1
column: x_ege_integer Nulls: 0
column: xegvy_integer Nulls: 0
column: y_ege_integer Nulls: 0
column: yegvx_integer Nulls: 0
column: class Nulls: 0
column: tag Nulls: 0




Habría que pensar si:
    1. Es válido/tiene sentido que pueda existir algún nulo en sus datos.
    2. Qué hacemos con estos nulos.
        - ¿Los imputamos?
        - ¿Los tiramos?
        - ¿De qué depende?

En nuestro caso, vamos a tirar el único caso con un nulo, por simplicidad.

In [10]:
# Respuesta

letters = letters.na.drop()



Tras remover todos los nulos, usamos el VectorAssembler con todas las variables excepto las objetivo (la nueva variable objetivo *tag* y la original *class*).

Asumimos aquí que todas las variables son numéricas y que las queremos usar como input para nuestro modelo.

In [11]:
# Respuesta

from pyspark.ml.feature import VectorAssembler

vectorassembler = VectorAssembler(inputCols=[_ for _ in letters.columns if _ not in  ('tag', 'class')], 
                                  outputCol='assembled_features')
letters = vectorassembler.transform(letters)



### Selección de variables



Vamos a hacer lo que vimos en el Notebook de selección de variables

Vamos a realizar la selección de variables en función de la importancia que le otorgue un algoritmo de ML.

Como el resultado de este algoritmo depende de la semilla que utilicemos, vamos a realizar varias simulaciones y a generar la media de los resultados en cada una.

In [17]:
# Respuesta

from pyspark.ml.classification import RandomForestClassifier
import random

random_seed = 4
num_iter = 10

random.seed(random_seed)

random_seeds=set([random.randint(0,10000) for _ in range(num_iter)])
features_random_seed = {}



Creamos un diccionario *features_random_seed* que es un diccionario de listas, donde cada lista contiene 1 elemento por semilla, es decir, uno por random forest. El obejtivo es quedarnos con las variables que explican el 95% de la importancia.

In [18]:
# Respuesta

for random_seed in random_seeds:
    rf = RandomForestClassifier(featuresCol=vectorassembler.getOutputCol(), labelCol='tag', seed = random_seed)
    rf_model = rf.fit(letters)
    
    importances = [(index, value) for index, value in enumerate(rf_model.featureImportances.toArray().tolist())]

    importances = sorted(importances, key=lambda value: value[1], reverse=True)

    imp = 0
    vector_assembler_cols = vectorassembler.getInputCols()
    for element in importances:
        feature = vector_assembler_cols[element[0]]
        importance = element[1]
        
        if imp < 0.95:
            features_random_seed[feature] = features_random_seed.get(feature, []) + [importance]
        else:
            features_random_seed[feature] = features_random_seed.get(feature, []) + [None]
        imp += element[1]

In [19]:
# Respuesta

features_random_seed = pd.DataFrame(features_random_seed).T
features_random_seed

,0,1,2,3,4,5,6,7,8,9
high_integer,None,None,None,None,None,None,None,None,None,None
onpix_integer,None,None,None,None,None,None,None,None,None,None
width_integer,0.115935,0.119533,0.0701563,0.11351,0.0986111,0.10889,0.127594,0.0915672,0.126174,0.0959411
x2bar_integer,0.112091,0.101648,0.0772696,0.0856666,0.109041,0.082472,0.0967184,0.0767635,0.0827652,0.0602689
x2ybr_integer,0.112876,0.0780184,0.0937907,0.0834113,0.07267,0.106973,0.0785156,0.0812151,0.0756823,0.105954
x_bar_integer,NaN,NaN,NaN,0.0179036,0.0251185,0.0201274,NaN,0.0512149,0.0231931,0.0250194
x_box_integer,None,None,None,None,None,None,None,None,None,None
x_ege_integer,0.11634,0.136102,0.104482,0.0715092,0.100809,0.0958737,0.0918505,0.120958,0.0651084,0.139627
xegvy_integer,0.070066,0.0245264,0.075306,0.0660088,0.0618353,0.0418931,0.0778766,0.0708447,0.0494945,0.0652668
xy2br_integer,0.160418,0.147171,0.176543,0.160391,0.199033,0.141797,0.177919,0.184016,0.203256,0.1528




Mostramos las variables más importantes. Las podemos tener en diccionario o en lista ordenada.

In [20]:
# Respuesta

feature_importances = features_random_seed.dropna(how='all').mean(axis=1)
feature_importances

width_integer    0.106791
x2bar_integer    0.088470
x2ybr_integer    0.088911
x_bar_integer    0.027096
x_ege_integer    0.104266
xegvy_integer    0.060312
xy2br_integer    0.170334
xybar_integer    0.031699
y2bar_integer    0.087022
y_bar_integer    0.142013
y_ege_integer    0.046880
yegvx_integer    0.026535
dtype: float64

In [23]:
# Respuesta

list_of_feature_importance = sorted(zip(feature_importances.index, feature_importances), 
                                    key=lambda x: x[1],
                                    reverse=True)
list_of_feature_importance

[('xy2br_integer', 0.17033443008368546),
 ('y_bar_integer', 0.14201330587864597),
 ('width_integer', 0.10679107744754845),
 ('x_ege_integer', 0.10426602229681688),
 ('x2ybr_integer', 0.08891062630343924),
 ('x2bar_integer', 0.0884704983869273),
 ('y2bar_integer', 0.08702226475991735),
 ('xegvy_integer', 0.06031182687195572),
 ('y_ege_integer', 0.046879765726505956),
 ('xybar_integer', 0.03169922602840045),
 ('x_bar_integer', 0.027096154545888443),
 ('yegvx_integer', 0.02653475020862006)]



Ahora mismo no necesitamos más que la lista sencilla.

In [26]:
# Respuesta

final_features = [_[0] for _ in list_of_feature_importance]

final_features

['xy2br_integer',
 'y_bar_integer',
 'width_integer',
 'x_ege_integer',
 'x2ybr_integer',
 'x2bar_integer',
 'y2bar_integer',
 'xegvy_integer',
 'y_ege_integer',
 'xybar_integer',
 'x_bar_integer',
 'yegvx_integer']



Hemos visto las variables más importantes, podríamos intentar usar sólo esas para nuestro modelo viendo cuál es el resultado de realizar eso. En esta caso, vamos a continuar con todas.

Estandarizamos los valores y lanzamos un modelo de clasificación dentro de un Pipeline

In [33]:
final_features = letters.columns
final_features.remove("tag")
final_features.remove("class")
final_features.remove("assembled_features")

In [34]:
final_features

['x_box_integer',
 'y_box_integer',
 'width_integer',
 'high_integer',
 'onpix_integer',
 'x_bar_integer',
 'y_bar_integer',
 'x2bar_integer',
 'y2bar_integer',
 'xybar_integer',
 'x2ybr_integer',
 'xy2br_integer',
 'x_ege_integer',
 'xegvy_integer',
 'y_ege_integer',
 'yegvx_integer']



**Regresión Logística** 

Vamos a hacer primero el vector assembler para alimentar este al StandardScaler, y la salida del mismo será el input para nuestro modelo.

Esto lo hacemos utilizando un Pipeline como ya vimos anteriormente (haciendo el fit y transform sólo en el pipeline).

In [35]:
# Respuesta

from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler
from pyspark.ml.classification import LogisticRegression

vector_assembler = VectorAssembler(inputCols=final_features, outputCol='assembled_important_features')
standard_scaler = StandardScaler(inputCol=vector_assembler.getOutputCol(), outputCol='standardized_features')
log_reg = LogisticRegression(featuresCol=standard_scaler.getOutputCol(), labelCol='tag')

pipeline_log_reg = Pipeline(stages=[vector_assembler, standard_scaler, log_reg])

letters_train, letters_test = letters.randomSplit([0.8,0.2], seed=4)

pipeline_model_log_reg = pipeline_log_reg.fit(letters_train)

letters_test_log_reg = pipeline_model_log_reg.transform(letters_test)

letters_test_log_reg.show(5)


+-------------+-------------+-------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-----+---+--------------------+----------------------------+---------------------+--------------------+--------------------+----------+
|x_box_integer|y_box_integer|width_integer|high_integer|onpix_integer|x_bar_integer|y_bar_integer|x2bar_integer|y2bar_integer|xybar_integer|x2ybr_integer|xy2br_integer|x_ege_integer|xegvy_integer|y_ege_integer|yegvx_integer|class|tag|  assembled_features|assembled_important_features|standardized_features|       rawPrediction|         probability|prediction|
+-------------+-------------+-------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-----+---+--------------------+-------------------------



**RandomForest**

Hacemos lo mismo que con la regresión logística pero utilizando como modelo el Random Forest.

In [37]:
# Respuesta

from pyspark.ml.classification import RandomForestClassifier

vector_assembler = VectorAssembler(inputCols=final_features, outputCol='assembled_important_features')
standard_scaler = StandardScaler(inputCol=vector_assembler.getOutputCol(), outputCol='standardized_features')
rf = RandomForestClassifier(featuresCol=standard_scaler.getOutputCol(), labelCol='tag')

pipeline = Pipeline(stages=[vector_assembler, standard_scaler, rf])

pipeline_model_rf = pipeline.fit(letters_train)

letters_test_rf = pipeline_model_rf.transform(letters_test)

letters_test_rf.show(5)

+-------------+-------------+-------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-----+---+--------------------+----------------------------+---------------------+--------------------+--------------------+----------+
|x_box_integer|y_box_integer|width_integer|high_integer|onpix_integer|x_bar_integer|y_bar_integer|x2bar_integer|y2bar_integer|xybar_integer|x2ybr_integer|xy2br_integer|x_ege_integer|xegvy_integer|y_ege_integer|yegvx_integer|class|tag|  assembled_features|assembled_important_features|standardized_features|       rawPrediction|         probability|prediction|
+-------------+-------------+-------------+------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-----+---+--------------------+-------------------------



### Evaluar modelos para decidir cuál predice mejor

In [38]:
# Respuesta

from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

def calculate_metrics(dataset, predictionCol, labelCol):
    metrics = BinaryClassificationEvaluator(rawPredictionCol=predictionCol, labelCol=labelCol)
    multimetrics = MulticlassClassificationEvaluator(predictionCol=predictionCol, labelCol=labelCol)

    # In binary case this four metrics will return the same value
    accuracy = multimetrics.evaluate(dataset, {metrics.metricName: "accuracy"})
    recall = multimetrics.evaluate(dataset, {metrics.metricName: "recall"})
    precision = multimetrics.evaluate(dataset, {metrics.metricName: "precision"})
    f1 = multimetrics.evaluate(dataset, {metrics.metricName: "f1"})

    area_under_pr = metrics.evaluate(dataset, {metrics.metricName: "areaUnderPR"})
    area_under_roc = metrics.evaluate(dataset, {metrics.metricName: "areaUnderROC"})
    
    return {'accuracy': accuracy, 
           'recall': recall, 
           'precision': precision,
           'f1': f1,
           'area_under_pr': area_under_pr, 
           'area_under_roc': area_under_roc}



### Regresión Logística

In [39]:
# Respuesta

calculate_metrics(letters_test_log_reg, 'prediction', 'tag')

{'accuracy': 0.7391654342039033,
 'area_under_pr': 0.391099414724887,
 'area_under_roc': 0.5407004432932595,
 'f1': 0.7391654342039033,
 'precision': 0.7391654342039033,
 'recall': 0.7391654342039033}



### Random Forest

In [40]:
# Respuesta

calculate_metrics(letters_test_rf, 'prediction', 'tag')

{'accuracy': 0.8216639150823668,
 'area_under_pr': 0.7147989771132053,
 'area_under_roc': 0.6463850974807331,
 'f1': 0.8216639150823668,
 'precision': 0.8216639150823668,
 'recall': 0.8216639150823668}